# Neural Machine Translation

## Drive Mount

In [ ]:
from google.colab import drive
import sys
sys.path.insert(0,'/content/drive/MyDrive/Neural Machine Translation/')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load tokenizer model For English and Spanish

In [ ]:
! python -m spacy download en_core_web_sm
! python -m spacy download es_core_news_sm
! pip install rouge
! pip install sentence_transformers

2022-08-08 13:58:36.199872: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2022-08-08 13:58:45.812593: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pyth

## Necessary Imports

In [ ]:
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import tensor
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer

from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

from sklearn.model_selection import train_test_split 
from rouge import Rouge
from testing import Testing
import math
import io 
import time
import torch

import pandas as pd


## Importing and Tokenizing Dataset

### Importing

In [ ]:
names = ['english', 'spanish', 'version_details']
dataset = pd.read_csv("/content/drive/MyDrive/Neural Machine Translation/Dataset/spa.txt", delimiter='\t', names=names)

### Splitting dataset into Train, Val and Test set

In [ ]:
en_train, en_test, es_train, es_test = train_test_split(dataset['english'], dataset['spanish'], test_size=0.2, random_state=1)
en_train, en_val, es_train, es_val = train_test_split(en_train, es_train, test_size=0.25, random_state=1) 

### Tokenizing

#### Loading Tokenizers

In [ ]:
es_tokenizer = get_tokenizer('spacy', language='es_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

#### Building Vocabulary and creating data tensors

In [ ]:
def build_vocabulary(data, tokenizer):
    counter = Counter()
    for datapoint in data:
        # print(datapoint)
        counter.update(tokenizer(datapoint))
    return vocab(counter, specials=['<unk>','<pad>','<bos>','<eos>'])

es_vocab = build_vocabulary(es_train, es_tokenizer)
en_vocab = build_vocabulary(en_train, en_tokenizer)

def process(en, es):
    data = []
    for raw_en, raw_es in zip(en,es):
        es_tensor = torch.tensor([es_vocab[token] if token in es_vocab else es_vocab['<unk>'] for token in es_tokenizer(raw_es)], dtype=torch.long)
        en_tensor = torch.tensor([en_vocab[token] if token in en_vocab else en_vocab['<unk>'] for token in en_tokenizer(raw_en)], dtype=torch.long)
        data.append((es_tensor, en_tensor))
    return data

train_data = process(en_train, es_train)
val_data = process(en_val, es_val)
test_data = process(en_test, es_test)

### Batch Generation and Padding sentences


In [ ]:
BATCH_SIZE = 128
PAD_IDX = es_vocab['<pad>']
BOS_IDX = es_vocab['<bos>']
EOS_IDX = es_vocab['<eos>']

In [ ]:
def batch_generator(data):
    es_batch, en_batch = [], []
    for es_item, en_item in data:
        es_batch.append(torch.cat([torch.tensor([BOS_IDX]), es_item, torch.tensor([EOS_IDX])]))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])]))
    es_batch = pad_sequence(es_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return es_batch, en_batch

train_iter = DataLoader(train_data,
    batch_size = BATCH_SIZE,
    shuffle = True, 
    collate_fn= batch_generator
    )

val_iter = DataLoader(val_data,
    batch_size = BATCH_SIZE,
    shuffle = True, 
    collate_fn= batch_generator
    )

test_iter = DataLoader(test_data,
    batch_size = BATCH_SIZE,
    shuffle = True, 
    collate_fn= batch_generator
    )

## Neural Network Model

### Global Parameters 

In [ ]:
src_vocab_len = len(es_vocab)
tgt_vocab_len = len(en_vocab)
EMB_SIZE = 512
NHEAD = 8
hdn_dim = 512
BATCH_SIZE = 128
num_encoder_layers= 3
num_decoder_layers = 3
epochs = 50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Transformer Model

In [ ]:
class Transformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_len: int, tgt_vocab_len: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Transformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.generator = nn.Linear(emb_size, tgt_vocab_len)
        self.src_tok_emb = TokenEmbedder(src_vocab_len, emb_size)
        self.tgt_tok_emb = TokenEmbedder(tgt_vocab_len, emb_size)
        self.positional_encoding = PositionalEncoderg(emb_size, dropout=dropout)

    def forward(self, src: tensor, trg: tensor, src_mask: tensor,
                tgt_mask: tensor, src_padding_mask: tensor,
                tgt_padding_mask: tensor, memory_key_padding_mask: tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: tensor, src_mask: tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: tensor, memory: tensor, tgt_mask: tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

### Token Embedding

In [ ]:
class TokenEmbedder(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

### Positional Encoding

In [ ]:
class PositionalEncoderg(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoderg, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_emb = torch.zeros((maxlen, emb_size))
        pos_emb[:, 0::2] = torch.sin(pos * den)
        pos_emb[:, 1::2] = torch.cos(pos * den)
        pos_emb = pos_emb.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_emb', pos_emb)

    def forward(self, token_embedding: tensor):
        return self.dropout(token_embedding +
                            self.pos_emb[:token_embedding.size(0),:])

### Mask Generation

In [ ]:
def subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

### Loss Function and Optimizer

In [ ]:
transformer = Transformer(NUM_ENCODER_LAYERS, num_decoder_layers,
                                 EMB_SIZE, src_vocab_len, tgt_vocab_len,
                                 hdn_dim)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.00001, betas=(0.9, 0.98), eps=1e-9
)

## Training and Evaluation

The Trained model is saved in models folder of parent folder NMT using Transformer Model




In [ ]:
def train(model, train_iter, optimizer):
    model.train()
    loss = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        loss += loss.item()
        
    torch.save(model, "/content/drive/MyDrive/Neural Machine Translation/NMT using Transformer Model/Models/t-nmt-model.pt")

    return loss / len(train_iter)


def evaluate(model, val_iter):
    model.eval()
    loss = 0
    for idx, (src, tgt) in (enumerate(val_iter)):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                       src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss += loss.item()
    return loss / len(val_iter)

In [ ]:
for epoch in range(1, epochs+1):
    start_time = time.time()
    train_loss = train(transformer, train_iter, optimizer)
    end_time = time.time()
    val_loss = evaluate(transformer, val_iter)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
           f"Epoch time = {(end_time - start_time):.3f}s"))       

Epoch: 1, Train loss: 6.827, Val loss: 5.309, Epoch time = 81.275s
Epoch: 2, Train loss: 4.950, Val loss: 4.590, Epoch time = 80.897s
Epoch: 3, Train loss: 4.436, Val loss: 4.171, Epoch time = 80.883s
Epoch: 4, Train loss: 4.088, Val loss: 3.874, Epoch time = 80.646s
Epoch: 5, Train loss: 3.833, Val loss: 3.651, Epoch time = 80.931s
Epoch: 6, Train loss: 3.632, Val loss: 3.462, Epoch time = 81.208s
Epoch: 7, Train loss: 3.461, Val loss: 3.307, Epoch time = 82.268s
Epoch: 8, Train loss: 3.311, Val loss: 3.160, Epoch time = 80.874s
Epoch: 9, Train loss: 3.179, Val loss: 3.040, Epoch time = 81.418s
Epoch: 10, Train loss: 3.058, Val loss: 2.933, Epoch time = 80.758s
Epoch: 11, Train loss: 2.949, Val loss: 2.825, Epoch time = 81.125s
Epoch: 12, Train loss: 2.847, Val loss: 2.737, Epoch time = 80.603s
Epoch: 13, Train loss: 2.751, Val loss: 2.650, Epoch time = 80.706s
Epoch: 14, Train loss: 2.662, Val loss: 2.568, Epoch time = 80.895s
Epoch: 15, Train loss: 2.579, Val loss: 2.490, Epoch time

## Translating Sentences from Spanish to English:


### Greedy Decode and Translation

In [ ]:
def decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys


def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):

    model.eval()
    tokens = [BOS_IDX] + [src_vocab.__getitem__(tok) if src_vocab.__contains__(tok) else src_vocab.__getitem__("<unk>") for tok in src_tokenizer(src)] + [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1))
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    # print()
    return " ".join([tgt_vocab.lookup_token(tok.item()) for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [ ]:
import re
from string import digits
def preprocess_for_nmt(sent):

  # convert text to lower case
  sent = sent.lower()

  # removing unnecessarily long spaces 
  sent = re.sub(" +", " ", sent)

  # removing quotes
  sent = re.sub("'", "", sent)

  # replacing digits with none: in this --> "str.maketrans('','', digits)" ------  returns a map for ascii codes of digits 0 through 9, that is to be replaced by None
  sent = sent.translate(str.maketrans('','', digits))

  # Adding spaces before and after punctuations (issue : The list of puctuation is from the reference I am using, when I add the list, it throws multiple errors)
  sent = re.sub(r"([?.!,¿])", r" \1 ", sent)

  # Strip the white spaces
  sent = sent.strip()

  return sent

In [ ]:
def load_model():
  model = torch.load('/content/drive/MyDrive/Neural Machine Translation/NMT using Transformer Model/Models/Final_model.pt', map_location=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))
  return model

## Testing

In [ ]:
model = load_model()
expected = en_test
pred = [translate( model , sentence, es_vocab, en_vocab, es_tokenizer ) for sentence in es_test]

In [ ]:

from testing import Testing
! pip install sentence_transformers
! pip install rouge
test = Testing(pred, expected)
test.score()
print("Precision: ",test.precision)
print("Recall Score: ",test.recall)
print("F1 Score: ",test.f1)
print("Bleu Score: ",test.bleu)

Precision: 0.6250791
Recall Score: 0.6424230
F1 Score: 0.6451164
Bleu Score: 0.5540225
